In [10]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from proj1_helpers import *
from data_helpers import *
from cross_validation import *
from plots import *

In [12]:
#LOAD TRAINING DATA
DATA_TRAIN_PATH = 'data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [13]:
#CLEAN TRAINING DATA
bounds = [0.0, 0.5]
tx_clean, ind_remov = treat_undefined_values(bounds, tX)
ind_remov

[4, 5, 6, 12, 26, 27, 28]

In [14]:
#MODEL BUILDING
tx, mean, std = standardize(tx_clean,0)
tx_augmented = build_poly_all_features(tx,8)
y,tx = build_model_data(tx_augmented,y)
y = classify(y)
num_samples = len(y)
num_features = tx.shape[1]
y = y.reshape(num_samples,1)


In [15]:
num_samples, num_features
tx.shape

(250000, 185)

In [16]:
from compute_gradient import *
from cost import *
from implementations import *

In [17]:
#Step-size

lambdas = np.logspace (-30,1,21)


#Regularization factor
# lambda_ = 0.01 142000 loss 10 000 iters
#lambda_ = 0.001 141000 loss 10 000 iters
#lambda_ = 0.02
#loss_min = np.inf
tx.shape,y.shape,y

((250000, 185), (250000, 1), array([[1.],
        [0.],
        [0.],
        ...,
        [1.],
        [0.],
        [0.]]))

In [18]:
losses_tr = np.zeros(len(lambdas))
losses_val = np.zeros(len(lambdas))

k_fold = 10
seed = 1
loss_min = np.inf
max_accuracy = 0

#CHOOSE LAMBDA AND GAMMA
for ind_lambda, lambda_ in enumerate(lambdas):
    
    loss_training = 0
    loss_validation = 0
    training_accuracy = 0
    validation_accuracy = 0
    for i,k in enumerate(range(k_fold)):
        tx_train, y_train, tx_val, y_val = cross_validation(y, tx, k, k_fold, seed)
        w, loss_train_k = ridge_regression(y_train, tx_train, lambda_)
        loss_val_k = ridge_loss (y_val,tx_val,w,lambda_)
        loss_training += loss_train_k
        loss_validation += loss_val_k
        training_accuracy += predict_accuracy(y_train,tx_train,w)
        validation_accuracy += predict_accuracy(y_val,tx_val,w)
        
    training_accuracy /= k_fold
    validation_accuracy /= k_fold
    loss_training /= k_fold
    loss_validation /= k_fold
    print(lambda_)
    print(loss_training)
    print(loss_validation)
    print(training_accuracy)
    print(validation_accuracy)
    if (validation_accuracy > max_accuracy):
        weights_star = w
        lambda_star = lambda_
        loss_min = loss_validation
        max_accuracy = validation_accuracy 
    losses_tr[ind_lambda] = loss_training
    losses_val[ind_lambda] = loss_validation



1e-30
4.524045292656571
24150542563113.945
70.83226666666667
70.69239999999999
3.54813389233576e-29
4.524075748226109
24150542563113.945
70.83231111111111
70.69239999999999
1.2589254117941712e-27
4.524045292768095
24150542563113.945
70.83226666666667
70.69239999999999
4.4668359215096167e-26
4.524045296616623
24150542563113.945
70.83226666666667
70.69239999999999
1.5848931924611108e-24
4.524045433167583
24150542563113.945
70.83226666666667
70.69239999999999
5.623413251903491e-23
4.524050278178466
24150542563113.945
70.83226666666667
70.69239999999999
1.9952623149688827e-21
4.52422218565171
24150542563113.945
70.83226666666667
70.69239999999999
7.079457843841402e-20
4.5303216929733425
24150542563113.953
70.83226666666667
70.69239999999999
2.5118864315095718e-18
4.746740379517642
24150542563114.168
70.83226666666667
70.69239999999999
8.912509381337514e-17
3.0506953052414607
24141312022550.633
72.62222222222223
72.71600000000001
3.1622776601683794e-15
3.1226375776373345
24117455438190.188


In [24]:
lambdas

array([1.00000000e-06, 6.81292069e-06, 4.64158883e-05, 3.16227766e-04,
       2.15443469e-03, 1.46779927e-02, 1.00000000e-01])

In [10]:
loss_min, lambda_star, max_accuracy

(19704880.660206415, 1.1220184543019653e-13, 79.2204)

In [19]:
#PREDICT TRAINING ACCURACY
training_accuracy = predict_accuracy(y,tx,weights_star)
training_accuracy

79.8368

In [198]:
#LOAD TEST SET
DATA_TEST_PATH = 'data/test.csv'
ytest, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [203]:
#CLEAN AND STANDARDIZE TEST SET
tX_test_clean = np.delete(tX_test, ind_remov, axis=1)
tX_test_clean.shape


(568238, 23)

In [206]:
tx_test_clean = remove_undefined_values (tX_test_clean)
tx_test_clean.shape

(568238, 23)

In [207]:
tx_test_clean, _, _ = standardize(tx_test_clean,0)


In [210]:
tx_test_augmented = build_poly_all_features(tx_test_clean,4)

(568238, 23)

In [213]:
tx_test_augmented.shape

(568238, 46)

In [214]:
#BUILD TEST MODEL
ytest, tx_test = build_model_data(tx_test_augmented,ytest)

In [215]:
#PREDICT LABELS
OUTPUT_PATH = 'data/submission_reg_logistic-degre5.csv'
y_pred = predict_labels(weights_star, tx_test,'logistic')
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)